In [33]:
%matplotlib qt
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
#import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import math
from PIL import Image

from openpyxl import load_workbook
import xlsxwriter

from itertools import *
from operator import *

In [34]:
min_fix = 0.100 # 100ms
min_cluster_size = 0.7 # cm
screenx = 1280
screeny = 1080
frame_res = 90.0

# Derived value(s)
min_fix_pts = round(min_fix * frame_res)



In [35]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    #'2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [36]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [37]:
TimeDwellOrig = 800
TimeFixation = 300

In [38]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [39]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [40]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [41]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dic_dwellTimeOrig_not800:
        TimeDwellOrig = dic_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [42]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)    
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    replacingList = []
    PhraseLogReduced = findAndRemoveTrials(session_name, dict_phraseStim, PhraseLogReduced, replacingList)
    
    
    return PhraseLogReduced

In [43]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [44]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [45]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [46]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [47]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
        
    return trials    

In [48]:
def FindExptStartEndTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            startTimeTrial = endTimeTrial + datetime.timedelta(seconds=5)
            timeTrialDict['start'].append(startTimeTrial)
            
            
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
    
          
    return timeTrialDict_reading, timeTrialDict_writing

In [49]:
def filterBlinks_wBlinkData(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    extraBlinkSamples = 18
    
    
    #pupilData_woSingleMissingData = pupilData.copy()
    #timeList_woSingleMissingData = timeInDatetime.copy()
    #timeInS_woSingleMissingData = timeInS_Trial[-1]
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single = np.argwhere(np.isnan(pupilDataL))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        interpolatedNan_bool = np.array([False]*len(pupilData_df['pupilLeft']))
        return pupilData_df, interpolatedNan_bool
    
    # find the index and values to replace for single nan values
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    
    interpolatedNan_bool = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    missingData_single = interpolatedNan_bool
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest = [i for i, x in enumerate(missingVal_Rest_trueFalse) if x]
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        return pupilData_df, interpolatedNan_bool

    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    
    
    
    # create a vector with True if a blink was at the position
    samplingFrequency = 90
    blinkLengthMax = np.ceil(0.5*samplingFrequency)
    blinkLengthMin = np.ceil(0.075*samplingFrequency)
    blinkStart_tupleList_wLength_allMissingData = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength]    
    blinkStart_tupleList_wLength_maxBlinkLength = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength if blinkLength<blinkLengthMax and blinkLength>blinkLengthMin]
    
    missingData_blinks_bool = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength_maxBlinkLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        missingData_blinks_bool[blinkIndices] = True
        
    
    
    missingData_blinks_boolAll = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        missingData_blinks_boolAll[blinkIndices] = True
        
    missingDataOverall_bool = missingData_single + missingData_blinks_boolAll
    
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    #blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    #timeInS_Trial_filter = timeInS_Trial[-1] - sum(blinkAndNonBlinkDurationList) 
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan_bool[replaceTrueList] = True
    
    
    
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan_bool, missingData_blinks_bool, missingDataOverall_bool, blinkStart_tupleList_wLength_maxBlinkLength, blinkStart_tupleList_wLength_allMissingData

In [50]:
def hampel(dvec, radius=5, nsigma=3, rem_nomed=False):

    # replace outliers with median values (hampel filter)
    
    mvec = pd.Series(dvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    svec = 1.4862 * np.abs(dvec-mvec).rolling(radius*2+1, center=True, min_periods=radius).median()
    plonk = np.abs(dvec-mvec) > nsigma*svec
    dvec = np.array(dvec)
    dvec[plonk.tolist()] = mvec[plonk.tolist()]

    # remove "bad data" where we cannot calculate a median value due to already missing values
    if (rem_nomed):
        dvec[np.isnan(mvec)] = np.nan
    return dvec

In [51]:
def filterBlinks(pupilDataL, pupilDataR, timeInDatetime):
    # filter any blinks and nan values lasting around 250ms (on average)
    # first the single nan occurances are replaced with mean of the values on either sides, 
    # as they are assumed to be from hardware problems
    # for the rest of the blinks, 250ms before and after the nan values are interpolated with a linear function
    # returns a dataframe with pupil size, and timestamp
    # http://faculty.washington.edu/chudler/facts.html
   
    
    # create a dataframe from the pupilsize and time
    pupilData_df = pd.DataFrame(list(zip(timeInDatetime, pupilDataL, pupilDataR)), columns=['timeStamp', 'pupilLeft', 'pupilRight'])
    
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (22 samples) before and after the blink will also be removed
    extraBlinkSamples = 18
    
    
    #pupilData_woSingleMissingData = pupilData.copy()
    #timeList_woSingleMissingData = timeInDatetime.copy()
    #timeInS_woSingleMissingData = timeInS_Trial[-1]
    
    # in case of single missing data, that are due to hardware error, replace with the mean of the pupil size before and
    # after the nan value
    # missing values will be the same for left and right pupil
    missingVal_Single = np.argwhere(np.isnan(pupilDataL))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list 
    
    
    
    # if no blinks present, return the data
    if len(missingVal_Single) == 0:
        interpolatedNan_bool = np.array([False]*len(pupilData_df['pupilLeft']))
        return pupilData_df, interpolatedNan_bool
    
    # find the index and values to replace for single nan values
    pupilData_tuples_replaceSingleNan_left = [(val, np.mean([pupilDataL[val-1], pupilDataL[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataL)-1)) if not np.isnan(pupilDataL[val-1]) and not np.isnan(pupilDataL[val+1])]
    pupilData_tuples_replaceSingleNan_right = [(val, np.mean([pupilDataR[val-1], pupilDataR[val+1]])) for i, val in enumerate(missingVal_Single) if (val != 0 and val != (len(pupilDataR)-1)) if not np.isnan(pupilDataR[val-1]) and not np.isnan(pupilDataR[val+1])]
    
    
    interpolatedNan_bool = np.array([True if ind in dict(pupilData_tuples_replaceSingleNan_left) else False for ind, val in enumerate(pupilDataL)])
    missingData_single = interpolatedNan_bool
    
    # replace the single nan values with the mean of the pupil size on either sides
    indList = -1
    for ind, val in pupilData_tuples_replaceSingleNan_left:
        indList = indList + 1
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilLeft')] = val
        pupilData_df.iloc[ind, pupilData_df.columns.get_loc('pupilRight')] = pupilData_tuples_replaceSingleNan_right[indList][1]
        
    
    # again, find the nan values in the pupil size
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index
    # to the next nan value
    
    
    
    # find the nan values again from pupilData['pupilLeft']
    missingVal_Rest_trueFalse = pupilData_df['pupilLeft'].isnull()
    missingVal_Rest = [i for i, x in enumerate(missingVal_Rest_trueFalse) if x]
    
    # if no blinks left, return the current pupilData
    if len(missingVal_Rest) == 0:
        return pupilData_df, interpolatedNan_bool

    
    # in the blinks left, find when the blinks start by finding a difference in the consecutive values of the indices
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    blinkStart_tupleList = [(ind, sum(missingVal_RestDifference[0:ind+1])) for ind, val in enumerate(missingVal_RestDifference) if val != 1]
    
    blinkStart_tupleList_wLength = list()
    
    # create a list of tuples of blink start index and the length of the blink
    ind = -1
    blinkLengthSum = 0
    for blink_ind, blinkStartInd in blinkStart_tupleList:
        ind = ind + 1
        if ind != len(blinkStart_tupleList) - 1:
            
            blinkLength = blinkStart_tupleList[ind+1][0]-blink_ind
            blinkLengthSum = blinkLengthSum + blinkLength
            
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
        else:
            # for the last blink -- all blink lengths summed and subtracted from the length of the list
            # missingVal_RestDifference 
            blinkLength = len(missingVal_RestDifference)-blinkLengthSum
            blinkStart_tupleList_wLength.append(tuple((blinkStartInd, blinkLength)))
     
    
    
    
    # create a vector with True if a blink was at the position
    samplingFrequency = 90
    blinkLengthMax = np.ceil(0.5*samplingFrequency)
    blinkLengthMin = np.ceil(0.075*samplingFrequency)
    blinkStart_tupleList_wLength_allMissingData = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength]    
    blinkStart_tupleList_wLength_maxBlinkLength = [(blinkStart, blinkLength) for blinkStart, blinkLength in blinkStart_tupleList_wLength if blinkLength<blinkLengthMax and blinkLength>blinkLengthMin]
    
    missingData_blinks_bool = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength_maxBlinkLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        missingData_blinks_bool[blinkIndices] = True
        
    
    
    missingData_blinks_boolAll = np.array([False]*len(pupilData_df['pupilLeft']))
    for blinkStart, blinkLength in blinkStart_tupleList_wLength:
        blinkIndices = np.arange(blinkStart, blinkStart + blinkLength).astype(int)
        missingData_blinks_boolAll[blinkIndices] = True
        
    missingDataOverall_bool = missingData_single + missingData_blinks_boolAll
    
    # add to vector with start and end of tuple
    #beforeAfterNan = [False]*len(pupilData_df['pupilLeft'])
    #for blinkStart, blinkLength in blinkStart_tupleList_wLength:
    #    beforeAfterNan[blinkStart] = True
    #    beforeAfterNan[blinkStart+blinkLength] = True
    #    #print('start and end points: ', pupilData_df['timeStamp'][blinkStart], pupilData_df['timeStamp'][blinkStart + blinkLength])
    
    
    # create lists with start and end values for the blinks, based on blinkStart_tupleList_wLength, regardless of the blink length
    blink_missingData_startList = [blinkStartInd - extraBlinkSamples if (blinkStartInd - extraBlinkSamples) > 0 else 0 for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    blink_missingData_endList = [blinkStartInd + blinkLength + extraBlinkSamples if (blinkStartInd + blinkLength + extraBlinkSamples) < (len(pupilData_df['pupilLeft'])-1) else (len(pupilData_df['pupilLeft'])-1) for blinkStartInd, blinkLength in blinkStart_tupleList_wLength]
    # create a list of tuples from the start and end points of the blink
    blink_missingData_startEndTuple = [(blinkStart, blink_missingData_endList[ind]) for ind, blinkStart in enumerate(blink_missingData_startList)] 
    
    
    # check if blinks need to be combined - blinksCombine is a list of list of 2 elements, the index of the blinks that should be combined
    blinksCombine = [[ind, ind+1] for ind, blink in enumerate(blink_missingData_startEndTuple[0:-1]) if blink[1] > blink_missingData_startEndTuple[ind+1][0]]
        
    if blinksCombine:
        # combine blinks that need to be combined - if multiple consecutive blinks need to be removed: eg - [1, 2], [2, 3] 
        # are included in the blinksCombine, the combined version should be [1, 3] 
        blinksCombineFinal = list()
        ind = -1
        while ind < len(blinksCombine)-2:
            
            ind = ind + 1
            blinkCombining = blinksCombine[ind]
            blinksCombineFinal.append(blinkCombining)
            while ind < len(blinksCombine)-2 and blinkCombining[1] == blinksCombine[ind+1][0]:
                # change the ending of the last added blink of blinksCombineFinal
                blinksCombineFinal[-1][1] = blinksCombine[ind+1][1]
                ind = ind + 1
            
            
        if len(blinksCombine) == 1:
            blinksCombineFinal = blinksCombine.copy()
            
        
        if blinksCombine[-1][1] != blinksCombineFinal[-1][1]:
            if blinksCombine[-1][0] == blinksCombineFinal[-1][1]:
                blinksCombineFinal[-1][1] = blinksCombine[-1][1]
            else:
                blinksCombineFinal.append(blinksCombine[-1])
        
        
        
        
    #    for w, z in groupby(sorted(list(blinksCombine)), lambda x, y=itertools.count(): next(y)-x):
    #        group = list(z)
    #        blinksCombineFinal.append(tuple((group[0], group[-1])))
        
        for x in sorted(blinksCombineFinal, reverse=True):
            new_start = blink_missingData_startEndTuple[x[0]][0] 
            new_end = blink_missingData_startEndTuple[x[1]][1] 
            
            x_start = x[0]
            x_end = x[1]
            
            # delete also the blinkStart_tupleList_wLength, since it is going to be used to compute other metrics
            for blinkRemove in range(x[1], x[0]-1, -1):
                del blink_missingData_startEndTuple[blinkRemove]
            
            blink_missingData_startEndTuple.insert(x[0], tuple((new_start, new_end)))
    
    
    #blinkAndNonBlinkDurationList = [length/90 for start, length in blinkStart_tupleList_wLength]
    #timeInS_Trial_filter = timeInS_Trial[-1] - sum(blinkAndNonBlinkDurationList) 
    
    
    # remove blinks from data
    for blinkStart, blinkEnd in blink_missingData_startEndTuple:
        pupilData_df.loc[blinkStart:blinkEnd,'pupilLeft'] = np.nan
        pupilData_df.loc[blinkStart:blinkEnd,'pupilRight'] = np.nan
        replaceTrueList = range(blinkStart, blinkEnd+1, 1)
        interpolatedNan_bool[replaceTrueList] = True
    
    
    
    pupilData_df['pupilLeft'] = pupilData_df['pupilLeft'].astype(float).interpolate('linear', limit_direction = 'both')
    pupilData_df['pupilRight'] = pupilData_df['pupilRight'].astype(float).interpolate('linear', limit_direction = 'both')
    
    if pupilData_df.isnull().any().any():
        print('nan values in filtered data')
        #for i,val in enumerate(pupilData_filter[0:5000]):
        #    print(i, val, pupilData_woSingleMissingData[i])
        
    
    return pupilData_df, interpolatedNan_bool

In [52]:
def FilterPupilSize(GazeLog, TimeTyping, subjectAndSessionName):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL_raw, pupilLogR_raw = Convert2ColumnSizesTo1(GazeLog, -5, -4, -2, -1)
    
    
    # reduce the data to start and end of typing time
    timeTyping_start, timeTyping_startInd = nearestTimePoint(timeGazeLog, TimeTyping['startTime'])
    timeTyping_end, timeTyping_endInd = nearestTimePoint(timeGazeLog, TimeTyping['endTime'])
    
    pupilLogL_wDefinedTime = pupilLogL_raw[timeTyping_startInd:timeTyping_endInd+1]
    pupilLogR_wDefinedTime = pupilLogR_raw[timeTyping_startInd:timeTyping_endInd+1]
    
    timeGazeLog_wDefinedTime = timeGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    
    timeInS_GazeLog_wDefinedTime = timeInternalGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    timeInS_Difference = [(t - s)/1000000 for s, t in zip(timeInS_GazeLog_wDefinedTime, timeInS_GazeLog_wDefinedTime[1:])]
    timeInS_Difference.insert(0, 0)
    
    
    timeInS = [sum(timeInS_Difference[:i]) for i, v in enumerate(timeInS_Difference)]
    
    
    pupilData_df, interpolated_items = filterBlinks(pupilLogL_raw['column1'], pupilLogR_raw['column1'], timeGazeLog)
    
    
    #timeGazeLog_plot = np.arange(0, timeInS[-1], 1/90)
    
    #plotPupilSize_checkFilter(pupilData_df, pupilLogL_wDefinedTime, blinkStartAndEnd, 'blink removal', subjectAndSessionName)
    
    
    
    
    pupilData_df_hampel = dict()
    
    pupilData_df_hampel = pupilData_df.copy()
    pupilData_df_hampel['pupilLeft'] = hampel(pupilData_df['pupilLeft'], 25, 3, False)
    pupilData_df_hampel['pupilRight'] = hampel(pupilData_df['pupilRight'], 25, 3, False)
        
    
        
    return pupilData_df_hampel

In [53]:
def FindReadingPartsOfTrial_inKeysSelected(EventTrials_reading, KeysSelected_new, full_path):
    # find the reading end in the pupil size data
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_reading['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        EventReading['start'].append(startTrialTime)
        EventReading_index['start'].append(startTrialInd)
        
        endTrialTime = EventTrials_reading['end'][ind]
        endTrialInd = KeysSelected_time.index(endTrialTime)
        
        keysSelected_trial = KeysSelected_keys[startTrialInd:endTrialInd]
        
        # compute the end of reading time -- which is taken when the keyboard with phrases key is pressed
        if session_folder_name not in dict_keyboardNotChange_ReadingTrials:
            
            endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
            endReading_keyTime = KeysSelected_time[endReading_keyInd]
            
        else:
            if ind == dict_keyboardNotChange_ReadingTrials[session_folder_name]:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('Sleep') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
            else:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
                
        EventReading['end'].append(KeysSelected_time[endReading_keyInd+1])
        EventReading_index['end'].append(endReading_keyInd+1)
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind], EventReading['end'][ind])
        
    
    
    return EventReading, EventReading_index

In [54]:
def FindWritingPartsOfTrial_inKeysSelected(EventTrials_writing, KeysSelected_new, EventReading):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_writing['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        endTrialTime = EventTrials_writing['end'][ind]
        endTimeReading = EventReading['end'][ind]
        
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTrialTime > endTimeReading:
            EventWriting['start'].append(startTrialTime)
            EventWriting_index['start'].append(startTrialInd)
        else:
            EventWriting['start'].append(endTimeReading)
            EventWriting_index['start'].append(KeysSelected_time.index(endTimeReading))
        
        EventWriting['end'].append(endTrialTime)
        EventWriting_index['end'].append(KeysSelected_time.index(endTrialTime))
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [55]:
def weightedMean(pupilData_df, window):
    
    # find the rolling standard deviations
    std_left_end = pupilData_df['pupilLeft'].rolling(window).std()[window-1:]
    std_right_end = pupilData_df['pupilRight'].rolling(window).std()[window-1:]
    
    std_left = [np.std(pupilData_df['pupilLeft'][0:i]) for i in range(1,window-1)]
    std_left.insert(0,0)
    
    
    std_right = [np.std(pupilData_df['pupilRight'][0:i]) for i in range(1,window-1)]
    std_right.insert(0,0)
    
    
        
    std_left.extend(std_left_end)
    std_right.extend(std_right_end)
    
    
    # if std_left is 0, the succeeding value is taken
    i = len(std_left)-1
    while i > -1:
        left = std_left[i]
        if left == 0:
            if i < len(std_left)-1:
                std_left[i] = std_left[i+1]
            else:
                std_left[i] = 2 # to give 0.5 for 1/std
        i = i - 1
    
    # if std_left is 0, the succeeding value is taken
    i = len(std_right) - 1
    while i > -1:
        right = std_right[i]
        if right == 0:
            if i < len(std_right)-1:
                std_right[i] = std_right[i+1]
                
            else:
                std_right[i] = 2 # to give 0.5 for 1/std
        i = i - 1
    
    # coefficients are 1/std
    coeff_left_notNormalized = 1/np.array(std_left)
    coeff_right_notNormalized = 1/np.array(std_right)
    
    coeff_left = coeff_left_notNormalized/(coeff_left_notNormalized+coeff_right_notNormalized)
    coeff_right_a = coeff_right_notNormalized/(coeff_left_notNormalized+coeff_right_notNormalized)
    coeff_right = 1-coeff_left
    
    #for i, x in enumerate(coeff_left):
    #    print(coeff_left_notNormalized[i], x, coeff_right_notNormalized[i], coeff_right[i])
    
    
    #coeff_left_notNormalized = [1/std if not np.isnan(std) and std!=0 else 0.5 for std in std_left]
    #coeff_right_notNormalized = [1/std if not np.isnan(std) and std!=0 else 0.5 for std in std_right]
    
    #coeff_left = np.array([left/(left+coeff_right_notNormalized[ind]) for ind, left in enumerate(coeff_left_notNormalized)])
    #coeff_right = 1-coeff_left
    
    pupilData_df['pupilMean_weighted'] = coeff_left*pupilData_df['pupilLeft'] + coeff_right*pupilData_df['pupilRight']
    pupilData_df['pupilMean'] = 0.5*pupilData_df['pupilLeft'] + 0.5*pupilData_df['pupilRight']
    
    #error = [np.abs(np.sum(pupilData_df['pupilLeft'][ind:ind+window-1])-np.sum(pupilData_df['pupilRight'][ind:ind+window-1]))/window for ind, left in enumerate(pupilData_df['pupilLeft'])]
    
    
    #print(len(pupilData_df['pupilLeft']), len(error))
    
    #print(error[0:20])
    #print(error[-20:])
    
    #print(np.corrcoef(pupilData_df['pupilLeft'], pupilData_df['pupilRight']))
    
    corr = np.corrcoef(pupilData_df['pupilLeft'], pupilData_df['pupilRight'])[0][1]
    
    
    #error = np.abs(pupilData_df['pupilLeft'] - pupilData_df['pupilRight'])
    
    #plt.figure()
    #plt.plot(error)
    #plt.plot(pupilData_df['pupilLeft'])
    #plt.plot(pupilData_df['pupilRight'])
    
    
    
    
    return pupilData_df, corr

In [56]:
def FilterPupilSize_wWeightedMean(GazeLog, TimeTyping, subjectAndSessionName):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupil_indWrtValidityL = [1, 2]
    pupil_validityL = 4
    pupilLogL_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityL, pupil_validityL)
    
    pupil_indWrtValidityR = [1, 2]
    pupil_validityR = 5
    pupilLogR_raw = gazeConvert2ColumnsTo1(GazeLog, pupil_indWrtValidityR, pupil_validityR)
    
    
    # reduce the data to start and end of typing time
    timeTyping_start, timeTyping_startInd = nearestTimePoint(timeGazeLog, TimeTyping['startTime'])
    timeTyping_end, timeTyping_endInd = nearestTimePoint(timeGazeLog, TimeTyping['endTime'])
    
    pupilLogL_wDefinedTime = pupilLogL_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    pupilLogR_wDefinedTime = pupilLogR_raw['column1'][timeTyping_startInd:timeTyping_endInd+1]
    
    timeGazeLog_wDefinedTime = timeGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    
    timeInS_GazeLog_wDefinedTime = timeInternalGazeLog[timeTyping_startInd:timeTyping_endInd+1]
    timeInS_Difference = [(t - s)/1000000 for s, t in zip(timeInS_GazeLog_wDefinedTime, timeInS_GazeLog_wDefinedTime[1:])]
    timeInS_Difference.insert(0, 0)
    
    
    #timeInS = [sum(timeInS_Difference[:i]) for i, v in enumerate(timeInS_Difference)]
    
    pupilData_df, interpolated_items = filterBlinks(pupilLogL_wDefinedTime, pupilLogR_wDefinedTime, timeGazeLog_wDefinedTime)
    
    
    
    
    #timeGazeLog_plot = np.arange(0, timeInS[-1], 1/90)
    
    #plotPupilSize_checkFilter(pupilData_df, pupilLogL_wDefinedTime, blinkStartAndEnd, 'blink removal', subjectAndSessionName)
    
    
    
    
    pupilData_df_hampel = dict()
    
    pupilData_df_hampel = pupilData_df.copy()
    pupilData_df_hampel['pupilLeft'] = hampel(pupilData_df['pupilLeft'], 25, 3, False)
    pupilData_df_hampel['pupilRight'] = hampel(pupilData_df['pupilRight'], 25, 3, False)
        
    
    # weighted mean of right and left pupils, based on their standard deviation 
    std_window = 25
    pupilDataMean_df, correlationRL = weightedMean(pupilData_df_hampel, std_window)
    
    
        
    return pupilDataMean_df, interpolated_items, correlationRL

In [57]:
def EventPartsFromPupilData(EventTimeInKeys, PupilSize_df, full_path):
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    EventTime = dict()    
    EventTime['start'] = list()
    EventTime['end'] = list()
    
    EventIndex = dict()    
    EventIndex['start'] = list()
    EventIndex['end'] = list()
    
    EventBaseline_startKeyTime = list()
    
    for ind, eventStartInKeys in enumerate(EventTimeInKeys['start']):
        
        eventStartTime, eventStartInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), eventStartInKeys)
        eventEndTime, eventEndInd = nearestTimePoint(PupilSize_df['timeStamp'].tolist(), EventTimeInKeys['end'][ind])
        
        # reading start is the same as trial start
        EventTime['start'].append(eventStartTime)
        EventIndex['start'].append(eventStartInd)
        
        EventTime['end'].append(eventEndTime)
        EventIndex['end'].append(eventEndInd)
        
        
    return EventTime, EventIndex    

In [58]:
def gazeConvert2ColumnsTo1(GazeLog, columnIndwValidity_list, indValidity):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    #columnInd_list = [joinColumn1_1, joinColumn1_2, joinColumn2_1, joinColumn2_2]
    
    # number of columns in the final dictionary
    nColumns = int(len(columnIndwValidity_list)/2)
    
    # dictionary of columns that are to be joined later
    columns_beforeDecimal = dict()
    columns_afterDecimal = dict()
    
    # dictionary of joined columns
    columnsFinal = dict()
    
    # dictionary to find and equalize missing values in every column
    missingVal_column = dict()
    missingVal = list()
    
    # find correct index of validity column to be used, to find the actual columns relative to that
    columnsValidity_inUse = list()
    
    for ind, row in enumerate(GazeLog):
        #print(ind)
        #print(sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])

        columnsValidity = (sorted(list(np.where(np.array(row) == 'Valid')[0])+list(np.where(np.array(row)=='Invalid')[0]))[indValidity])
        columnsValidity_inUse.append(int(columnsValidity))
    
    columnsValidity_inUse = np.array(columnsValidity_inUse)
    
    columnInd_list = [[columnsValidity_inUse+i] for i in columnIndwValidity_list]
    
    for ind in range(0, nColumns):
        
        dict_name = 'column' + str(ind+1)
        columnsFinal[dict_name] = list()
        columns_afterDecimal[dict_name] = list()
                
        #for indItem, item4 in enumerate(GazeLog):
        #    if 'Invalid' not in item4:
        #        if columnInd_list[2*ind+1][0][indItem] < len(item4):
        #            columns_afterDecimal[dict_name].append(item4[columnInd_list[2*ind+1][0][indItem]])
        #        else:
        #            columns_afterDecimal[dict_name].append('0')
        #    else:
        #        columns_afterDecimal[dict_name].append('nan')
        
                
        columns_beforeDecimal[dict_name] = [item4[columnInd_list[2*ind][0][indItem]] if 'Invalid' not in item4 else 'nan' for indItem, item4 in enumerate(GazeLog)]
        columns_afterDecimal[dict_name] = [item4[columnInd_list[2*ind+1][0][indItem]] if 'Invalid' not in item4 and columnInd_list[2*ind+1][0][indItem] < len(item4) else 'nan' for indItem, item4 in enumerate(GazeLog)]
        
        
        for i in range(0, len(columns_beforeDecimal[dict_name])):
            if 'Valid' not in columns_beforeDecimal[dict_name][i] and 'Valid' not in columns_afterDecimal[dict_name][i]:
                if 'nan' not in columns_beforeDecimal[dict_name][i] and 'nan' not in columns_afterDecimal[dict_name][i]:
                    if float(columns_afterDecimal[dict_name][i]) > 0: 
                        columnsFinal[dict_name].append(float(columns_beforeDecimal[dict_name][i]+'.'+columns_afterDecimal[dict_name][i]))
                    else:
                        columnsFinal[dict_name].append(np.nan)
                else:
                    columnsFinal[dict_name].append(np.nan)
            else:
                # Rarely, the pupil size is a whole number
                columnsFinal[dict_name].append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
                # right or left eye has whole number pupil size
    
        missingVal_column[dict_name] = np.argwhere(np.isnan(columnsFinal[dict_name]))
        missingVal_column[dict_name] = list(itertools.chain.from_iterable(missingVal_column[dict_name])) # flatten the list
        
        missingVal.extend(missingVal_column[dict_name])
        
        
    
    missingVal = sorted(set(missingVal))
    
    
    # if one of the columns are nan, the other one is converted too
    for column in range(0, nColumns):
        dict_name = 'column' + str(column+1)
        for ind in missingVal:
            if ind < len(columnsFinal[dict_name]):
                columnsFinal[dict_name][ind] = np.nan
                
    
    
    
    #print(len(columnsFinal['column1']), len(columnsFinal['column2']))
    
    
    

    return columnsFinal

In [59]:
def getGazePointInDegrees(GazeLog):

    # Position of eyes in the UCS by Tobii (in mm)
    #print('gazeOrigin')
    gazeOriginUCS_validityL = 0
    gazeOriginUCS_indWrtValidityL = [1, 2, 3, 4, 5, 6]
    gazeOriginUCS_Left = gazeConvert2ColumnsTo1(GazeLog, gazeOriginUCS_indWrtValidityL, gazeOriginUCS_validityL)
    
    gazeOriginUCS_validityR = 1
    gazeOriginUCS_indWrtValidityR = [1, 2, 3, 4, 5, 6]
    gazeOriginUCS_Right = gazeConvert2ColumnsTo1(GazeLog, gazeOriginUCS_indWrtValidityR, gazeOriginUCS_validityR)
    
    # find the average of each to compute the gaze origin
    gazeOriginUCS = dict()
    gazeOriginUCS['x'] = np.array([(v+gazeOriginUCS_Right['column1'][i])/2 for i, v in enumerate(gazeOriginUCS_Left['column1'])])
    gazeOriginUCS['y'] = np.array([(v+gazeOriginUCS_Right['column2'][i])/2 for i, v in enumerate(gazeOriginUCS_Left['column2'])])
    gazeOriginUCS['z'] = np.array([(v+gazeOriginUCS_Right['column3'][i])/2 for i, v in enumerate(gazeOriginUCS_Left['column3'])])
    
    
    # Location of eye gaze on screen in the UCS by Tobii (in mm)
    #print('gazePoint')
    gazePointUCS_validityL = 2
    gazePointUCS_indWrtValidityL = [1, 2, 3, 4, 5, 6]
    gazePointUCS_Left = gazeConvert2ColumnsTo1(GazeLog, gazePointUCS_indWrtValidityL, gazePointUCS_validityL)
    
    gazePointUCS_validityR = 3
    gazePointUCS_indWrtValidityR = [1, 2, 3, 4, 5, 6]
    gazePointUCS_Right = gazeConvert2ColumnsTo1(GazeLog, gazePointUCS_indWrtValidityR, gazePointUCS_validityR)
    
    # find the average of gaze point in UCS
    gazePointUCS = dict()
    gazePointUCS['x'] = np.array([(v+gazePointUCS_Right['column1'][i])/2 for i, v in enumerate(gazePointUCS_Left['column1'])])
    gazePointUCS['y'] = np.array([(v+gazePointUCS_Right['column2'][i])/2 for i, v in enumerate(gazePointUCS_Left['column2'])])
    gazePointUCS['z'] = np.array([(v+gazePointUCS_Right['column3'][i])/2 for i, v in enumerate(gazePointUCS_Left['column3'])])
    
    # find the distance between the screen and eyes, dividing by 10 to get cm from mm 
    distanceEyeGaze = [(np.sqrt((gazePointUCS['x'][i]-gazeOriginUCS['x'][i])**2 + (gazePointUCS['y'][i]-\
                        gazeOriginUCS['y'][i])**2 + (gazePointUCS['z'][i]-gazeOriginUCS['z'][i])**2))/10 for i, v in\
                       enumerate(gazePointUCS['x'])]
    
    #plt.figure()
    #plt.plot(distanceEyeGaze)
    #plt.title('distance between eye and point of gaze')
    
    # Location of eye gaze on screen in the ADCS by Tobii (in arbitrary units)
    #print('gaze point on screen')
    gazePointADCS_indWrtValidityL = [-4, -3, -2, -1]
    gazePointADCS_validityL = 3
    gazePointADCS_Left_au = gazeConvert2ColumnsTo1(GazeLog, gazePointADCS_indWrtValidityL, gazePointADCS_validityL)
    
    
    gazePointADCS_indWrtValidityR = [-4, -3, -2, -1]
    gazePointADCS_validityR = 4
    gazePointADCS_Right_au = gazeConvert2ColumnsTo1(GazeLog, gazePointADCS_indWrtValidityR, gazePointADCS_validityR)
    
    
    # gazePointADCS is in arbitrary units and needs to be converted to cm  
    screenLength = 59
    screenWidth = 34.5
    
    gazePointADCS_Left_au['column1'] = [i*59 for i in gazePointADCS_Left_au['column1']]
    gazePointADCS_Left_au['column2'] = [i*34.5 for i in gazePointADCS_Left_au['column2']]
    
    gazePointADCS_Right_au['column1'] = [i*59 for i in gazePointADCS_Right_au['column1']]
    gazePointADCS_Right_au['column2'] = [i*34.5 for i in gazePointADCS_Right_au['column2']]
    
    gazePointADCS = dict()
    gazePointADCS['x'] = [(v+gazePointADCS_Right_au['column1'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column1'])]
    gazePointADCS['y'] = [(v+gazePointADCS_Right_au['column2'][i])/2 for i, v in enumerate(gazePointADCS_Left_au['column2'])]
    
    
    gazePoint_degrees = dict()
    gazePoint_degrees['x'] = np.array([v/distanceEyeGaze[i] for i, v in enumerate(gazePointADCS['x'])])
    gazePoint_degrees['y'] = np.array([v/distanceEyeGaze[i] for i, v in enumerate(gazePointADCS['y'])])
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = np.array(timeConversion(timeStrGazeLog))
    
    gazePoint_degrees['timeStamp'] = timeGazeLog
    
    
    return gazePoint_degrees, gazePointADCS, distanceEyeGaze

In [60]:
def spat_dist(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1]) # degrees 

In [162]:
def saccade_detection_velocityThreshold(gazeData, distanceFromScreen, vel_threshold):
    
    x_coord = np.array(gazeData['x'])
    y_coord = np.array(gazeData['y'])
    z_coord = np.array(distanceFromScreen)

        
    #calculate angular velocity
    angle = np.empty((len(distanceFromScreen),1))
    
    angle[0:4]= 0; #% BUffer of samples needed to calculate angular velocity
        
    for k in range(5,len(distanceFromScreen)):
            
        P = np.vstack([x_coord[k], y_coord[k], z_coord[k]]) # See Duchowsky (2003)
        Q = np.vstack([x_coord[k-5], y_coord[k-5], z_coord[k-5]]) # See Duchowsky (2003)
        #Q = [x_coord(k-5) y_coord(k-5) z_coord(k-5)];
        vect = np.sum(np.matmul(P,np.transpose(Q)))
        modul = np.sqrt(sum(P**2))*np.sqrt(sum(P**2))
        print((vect/modul)[0])
        #angle[k] = np.real(math.acos(vect/modul))
            
    
        
    v = angle*(1/(5/fs)); # velocity
        
    print(v)
    
    #Find samples that meet saccade criterion
    sacc_idx = list(np.where(abs(v)>vel_threshold)) + list(print(np.where(np.isnan(v))))
    sacc_idx.sort()
    
    print(sacc_idx)
    
    #sacc_idx = [find(abs(v) > vel_threshold); find(np.isnan(v))];
    #sacc_idx = sort(sacc_idx);


In [163]:
def cluster_frames_DBSCAN(npeyeframes):
    '''
    Cluster frames with a derivate of DBSCAN taking into account the time
    
    This DBSCAN variation uses DBSCAN in the local spatial neighbourhood with timing constraints:
    Single or a few outliers followed by additional connected points are considered noise, 
    and more points can be added to the current cluster
    However if enough disjoint points are seen that could make up a cluster (temporally!) it ends the current cluster
    '''

    def query_region(frames, first, reference):
        '''returns absolute indices of frames in neighbourhood of reference, from first to break/len(frames)'''
        i = first
        region = set()
        seq_outside = 0
        while i < len(frames):
            if spat_dist(frames[reference], frames[i]) < min_cluster_size:
                region.add(i)
                seq_outside = 0
            else:
                if i > reference:
                    seq_outside += 1
                    if seq_outside >= min_fix_pts:
                        break
            i += 1
        return region


    # first create labels; 0 means unassigned (so far)
    labels = np.zeros(len(npeyeframes), dtype=int)

    label = 1
    unassigned = 0
    nextp = unassigned
    while nextp < len(labels):
        #print ("DBSCAN index: " , nextp)
        if labels[nextp] == 0:
            neighbours = query_region(npeyeframes, unassigned, nextp)
            if len(neighbours) < min_fix_pts:
                #print ("   Too few neighbours; leaving point alone: %d" % len(neighbours))
                a = 2
            else:
                #print ("   New cluster: %d [Initial: %d]" % (label, len(neighbours)))
                unvisited = neighbours
                unvisited.remove(nextp)
                visited = set()
                visited.add(nextp)

                while len(unvisited) > 0:
                    first = unvisited.pop()

                    visited.add(first)
                    neighbours = query_region(npeyeframes, unassigned, first)
                    if len(neighbours) >= min_fix_pts:  # SHOULD THIS BE > 0 only?
                        unvisited.update(neighbours - visited)

                for v in visited:
                    assert labels[v] == 0, "Reassigning point to new cluster, should not take place"
                    labels[v] = label
                    nextp = max(nextp, v)

                unassigned = nextp + 1
                label += 1
                #print ("   Next unassigned: %d [Final: %d]" % (unassigned, len(visited)))
        #else:
            #print ("   Already assigned to %d" % labels[nextp])

        nextp += 1

    # remove any empty last cluster
    if len(npeyeframes[labels==label])==0:
        #print ("   Last cluster (%d) is empty, removing" % (label))
        label -= 1

    # calculate the cluster centers...
    #print ("   Ended up with %d clusters" % label)
    cluster_centers = np.empty([label+1, 3*4])
    
    
    for l in range(label+1):
        #print ("   Fixing frames in  clusters %d" % l)

        myframes = npeyeframes[labels==l]
        mylen = len(myframes)
        if (mylen > 0):
            
            cmean = np.mean(myframes, 0)
            cmin = np.amin(myframes, 0)
            cmax = np.amax(myframes, 0)
            cstd = np.std(myframes, 0)
            cluster_centers[l] = np.concatenate((cmean, cmin, cmax, cstd))
            #print(cmean, cmin, cmax, cstd)
        #print('')
        
        #fig, ax = plt.subplots(figsize = (20,15)) #, dpi = dpi)
        #plt.axis([min(min(gazePointBetweenTrial_x), 0), max(max(gazePointBetweenTrial_x), screenLength), \
        #          max(max(gazePointBetweenTrial_y), 0), min(min(gazePointBetweenTrial_y), screenWidth)])  
        
        #plt.axis([0, 60, 0, 40])
    
    
        
        
    
    return (labels, cluster_centers)

In [164]:
def cluster_frames(npeyeframes):
    '''
    Cluster the npeyeframes that each have (x,y,t) positions; need not be equitemporally sampled
    
    Returns a tupple of labels and centers
    where len(labels)==len(npeyeframes), and labels[i] is index into clusters for frame i
    and centers contains the (x,y,t) coordinates of each cluster found
    '''

    return cluster_frames_DBSCAN(npeyeframes)

In [165]:
def saccadeDetection_mDBSCAN(gazePoint_cm, gazePoint_degrees):
    
    gazePoint = np.array([(float(gazePoint_cm['x'][i]), float(gazePoint_cm['y'][i]), i/90) for i in \
                          range(len(gazePoint_cm['x'])) if not np.isnan(gazePoint_cm['x'][i]) ])
    gazePointInd = np.array([i for i in range(len(gazePoint_cm['x'])) if not np.isnan(gazePoint_cm['x'][i])])
    
    # Cluster the data into and centers and labels
    labels, cluster_centers = cluster_frames(gazePoint)
    
    # combine labels with nan
    labels_all = np.array([np.nan]*len(gazePoint_cm['x']))
    labels_all[gazePointInd] = labels
    
    #for i in cluster_centers:
    #    print(i)
        #print(i[0], i[2], abs(i[2] - i[0]))
    
    
    
    colorList = ['red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon', \
                'red', 'olivedrab', 'paleturquoise', 'plum', 'maroon', 'orange', 'lightgreen', 'skyblue', 'deeppink', \
                'chocolate', 'forestgreen', 'aqua', 'blueviolet', 'brown', 'hotpink', 'tomato', 'lime', 'teal', \
                'darkmagenta', 'sandybrown', 'firebrick', 'darkcyan', 'palevioletred', 'springgreen', 'salmon']
    
    window_time = 15
    sampling_frequency = 90
    window_start = 0
    # plot separate figures for every 30 s, to make it clear what is happening
    for ind_window in range(1, int(len(gazePoint_cm['x'])/window_time)):
        if window_start > len(gazePoint_cm['x']):
            break
        window_end = window_start + (window_time*sampling_frequency)
        if window_end > len(gazePoint_cm['x'])-1:
            window_end = len(gazePoint_cm['x'])-1
        if window_start == window_end:
            break
        #print(window_start, window_end)
        # check window, if it ends properly in either a fixation or a saccade
        # check current ending label:
        if not np.isnan(labels_all[window_end]):
            if labels_all[window_end] > 0:
                label_check = labels_all[window_end]
                for ind2 in range(window_end+1, len(labels_all)):
                    if labels_all[ind2] != label_check:
                        window_end_new = ind2-1
                        break
                        
            else:
                window_end_new = window_end
        else:
            window_end_new = window_end
                
        
        plt.style.use('dark_background')
        im = np.array(Image.open('keyboard_baselineQ.png'), dtype=np.uint8)
        fig, ax = plt.subplots(figsize = (20,15)) #, dpi = dpi)
        ax.imshow(im, extent=(0, 59, 34.5, 0))
        ax.axis([0, 60, 35, 0])
        p = list()
        label_n = 0
        labelPrevious = 0
        
        print(window_start, window_end_new, gazePoint_degrees['timeStamp'][window_start], gazePoint_degrees['timeStamp']\
              [window_end])
        
        # save the data for this window, to give to Helia
        timeToSave = range(0,window_end_new-window_start)
        gazePointX = gazePoint_cm['x'][window_start:window_end_new]
        gazePointY = gazePoint_cm['y'][window_start:window_end_new]
        labelsSave = labels_all[window_start:window_end_new]
        df_save = pd.DataFrame(list(zip(timeToSave, gazePointX, gazePointY, labelsSave)), columns = ['time', 'x', 'y', \
                                                                                                    'label'])
        
        
        for i, key in enumerate(range(window_start,window_end_new)):
            label = labels_all[key]
            if not np.isnan(label): # current point is not a blink
                if label == 0: # it is a saccade
                    ax.plot(gazePoint_cm['x'][key], gazePoint_cm['y'][key], marker = 'o', color = 'white')
                else: # point is a fixation
                    
                    labelCurrent = label
                    if labelCurrent != labelPrevious:
                        
                        label_n = label_n + 1
                        #print(i, label_n, colorList[label_n], gazePoint_cm['x'][i], gazePoint_cm['y'][i])
                        ax.text(gazePoint_cm['x'][key], gazePoint_cm['y'][key], label_n, horizontalalignment='center',\
                                size='large', color='white', weight='semibold')
            
                    
                        
                    ax.plot(gazePoint_cm['x'][key], gazePoint_cm['y'][key], marker = 'o', color = colorList[label_n])
                    labelPrevious = label
        plt.tight_layout()
        fig.savefig(r'C:\DTU\Results\201901_Expt\saccade\baselineQ_fixationDispersion_0p7cm.png', dpi = 300, \
                    bbox_to_anchor = (0.95, 0.1))
        
            #print(label_n)
        break
        window_start = window_end_new
    
    
    
    return df_save, window_start, window_end_new
    
    """
    
    #print('labels all: ', labels_all)
    
    #print('')
    # saccade computation from labels
    saccadeAmplitude, saccadeSamples = 0,0
    saccadeAmplitudeList, saccadeDurationList, saccadeVelocityList  = list(), list(), list()
    saccadeAvgAmplitudeList, saccadeAvgDurationList, saccadeAvgVelocityList, saccadePeakVelocityList = list(), list(), \
    list(), list()
    
    indexSaccade = list()
    
    amplitudeList, durationList, velocityList = list(), list(), list()

    for i,l in enumerate(labels_all[1:-1]):
        if not np.isnan(l):# current point is not a nan value
            if l == 0: # current point is not clustered            
                saccadeSamples = saccadeSamples + 1 
                #print(saccadeSamples, 'not nan: ')
                if not np.isnan(labels_all[i]): # checking the previous point if not blink
                    point_current = (gazePoint_degrees['x'][i+1], gazePoint_degrees['y'][i+1])
                    point_previous = (gazePoint_degrees['x'][i], gazePoint_degrees['y'][i])
                    saccadeAmplitude = saccadeAmplitude + spat_dist(point_current, point_previous)
                    
                    saccadeAmplitudeList.append(saccadeAmplitude)
                    saccadeDurationList.append(saccadeSamples/frame_res)
                    saccadeVelocityList.append(saccadeAmplitude*frame_res/saccadeSamples)
                    
                    amplitudeList.append(saccadeAmplitude)
                    durationList.append(saccadeSamples/frame_res)
                    velocityList.append(saccadeAmplitude*frame_res/saccadeSamples)
                    
                    indexSaccade.append(i+1)
                    #print(saccadeAmplitude, saccadeAmplitude*frame_res/saccadeSamples)
                if not np.isnan(labels_all[i+2]): # checking the subsequent point if not blink
                    
                    if labels_all[i+2] > 0: # if subsequent point is clustered # it is a fixation
                        point_current = (gazePoint_degrees['x'][i+1], gazePoint_degrees['y'][i+1])
                        point_next = (gazePoint_degrees['x'][i+2], gazePoint_degrees['y'][i+2])
                        
                        saccadeAmplitude = saccadeAmplitude + spat_dist(point_current, point_next)
                        saccadeAmplitudeList.append(saccadeAmplitude)
                        saccadeDurationList.append(saccadeSamples/frame_res)
                        saccadeVelocityList.append(saccadeAmplitude*frame_res/saccadeSamples)
                        
                        indexSaccade.append(i+2)
                        
                        amplitudeList.append(saccadeAmplitude)
                        durationList.append(saccadeSamples/frame_res)
                        velocityList.append(saccadeAmplitude*frame_res/saccadeSamples)
                    
                        # average velocity, etc:
                        saccadeAvgAmplitudeList.append(np.mean(amplitudeList))
                        saccadeAvgDurationList.append(np.mean(durationList))
                        saccadeAvgVelocityList.append(np.mean(velocityList))
                        saccadePeakVelocityList.append(np.max(velocityList))
                        
                        amplitudeList, durationList, velocityList = list(), list(), list()
                        
                        #print(saccadeAmplitude, saccadeAmplitude*frame_res/saccadeSamples, saccadeSamples/frame_res)
                        saccadeAmplitude, saccadeSamples = 0, 0
                        
                else:
                    # next point is a blink, the saccade should stop  
                        
                    if len(amplitudeList) > 0:
                        if len(amplitudeList) > 1:
                            # average velocity, etc:
                            saccadeAvgAmplitudeList.append(np.mean(amplitudeList))
                            saccadeAvgDurationList.append(np.mean(durationList))
                            saccadeAvgVelocityList.append(np.mean(velocityList))
                            saccadePeakVelocityList.append(np.max(velocityList))
                        else:
                            saccadeAvgAmplitudeList.append(amplitudeList[0])
                            saccadeAvgDurationList.append(durationList[0])
                            saccadeAvgVelocityList.append(velocityList[0])
                            saccadePeakVelocityList.append(velocityList[0])
                            
                    
                    amplitudeList, durationList, velocityList = list(), list(), list()
                    
                    
                    
                    #print('fixation/saccade stops here: ', saccadeAmplitude, saccadeSamples/frame_res)
                    
                    saccadeAmplitude, saccadeSamples = 0, 0
                    
    timeStamp = gazePoint_degrees['timeStamp'].tolist()     
    # obtain timestamp from indexSaccade
    timeStamp_saccades = [timeStamp[i] for i in indexSaccade]
    
    
    #print(len(saccadeAmplitudeList), len(saccadeDurationList), len(saccadeVelocityList))     
    #print(len(saccadeAvgAmplitudeList), len(saccadeAvgDurationList), len(saccadeAvgVelocityList), \
    #      len(saccadePeakVelocityList))
    
    
    return saccadeAmplitudeList, saccadeDurationList, saccadeVelocityList, timeStamp_saccades, saccadeAvgAmplitudeList, \
saccadeAvgDurationList, saccadeAvgVelocityList, saccadePeakVelocityList
"""

In [166]:
class DataForEverySubject:
    
    timeStamp = ''
    saccadeAmplitude = ''
    saccadeDuration = ''
    saccadeVelocity = ''
    
    
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip(self.timeStamp, self.saccadeAmplitude, self.saccadeDuration, \
                                          self.saccadeVelocity)), columns=['timeStamp', 'amplitude', 'duration', \
                                          'velocity'])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip(self.timeStamp, self.saccadeAmplitude, self.saccadeDuration, \
                                          self.saccadeVelocity)), columns=['timeStamp', 'amplitude', 'duration', \
                                          'velocity'])

        #workbook = xlsxwriter.Workbook(self.resultPathName)
        #book = workbook.add_worksheet()
        
        #book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='xlsxwriter')
        #writer.book = book
        #writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        
        dataFrame.to_excel(writer, index = False, header = False)
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        writer.save()
    
    

In [167]:
class DataForEverySubject_Average:
    
    
    saccadeAmplitude = ''
    saccadeDuration = ''
    saccadeVelocity = ''
    saccadePeakVelocity = ''
    
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip(self.saccadeAmplitude, self.saccadeDuration, self.saccadeVelocity, \
                                         self.saccadePeakVelocity)), columns=['amplitude', 'duration', 'velocity', \
                                                                              'peakVelocity'])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip(self.saccadeAmplitude, self.saccadeDuration, self.saccadeVelocity, \
                                         self.saccadePeakVelocity)), columns=['amplitude', 'duration', 'velocity', \
                                                                              'peakVelocity'])
        
        writer = pd.ExcelWriter(self.resultPathName, engine='xlsxwriter')
        
        dataFrame.to_excel(writer, index = False, header = False)
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        writer.save()
    
    
    

In [168]:


dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4



for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have 
            #gaze log and have been marked as notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        if '2019-02-11-11-18-30_1' not in root:
            continue
        
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
                
            print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            
            resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subjects\Saccades\\2DegreeFixationRadius' + \
            '\\' + subjAndSessionName +  '.xlsx'
            #resultFileName_average = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subjects\SaccadesAverage' + '\\' + \
            #subjAndSessionName +  '.xlsx'
            
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find trial times for writing
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root)
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, \
                                                   eventTrialsInKeysSelected_reading)
            
            sessionFolderName = root.split('\\')[-1]
            if sessionFolderName in dict_noGazeData:
                print('no gaze data present')
                saccadeAmplitude = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                saccadeDuration = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                saccadeVelocity = [np.nan]*len(eventTrialsInKeysSelected_writing['start'])
                continue
            
            
            # filter the data
            pupilMean_df, nanValues, correlationRL = FilterPupilSize_wWeightedMean(gazeLog, timeTyping, \
                                                                                       subjAndSessionName)
            
            
            # take the data from pupil size
            eventWriting, eventWriting_index = EventPartsFromPupilData(eventTrialsInKeysSelected_writing, \
                                                                           pupilMean_df, root)
            
            # detect saccades using Per's method
            gazePoint_degrees, gazePoint_cm, distanceFromScreen = getGazePointInDegrees(gazeLog)
            
            saccade_detection_velocityThreshold(gazePoint_cm, distanceFromScreen, vel_threshold=3)
            
            #saccadeAmplitude, saccadeDuration, saccadeVelocity, saccadeTimeStamp, saccadeAvgAmplitude, saccadeAvgDuration,\
            df_save, window_start, window_end = saccadeDetection_mDBSCAN(gazePoint_cm, gazePoint_degrees)
            
            pupil_save = pupilMean_df['pupilMean_weighted'][window_start:window_end]
            df_save['pupil'] = pupil_save
            cols = df_save.columns.tolist()
            
            cols_new = cols[0:3] + [cols[4]] + [cols[3]]
            df_save = df_save[cols_new]
            
            #df_save.to_csv(r'C:\DTU\PupilProcessing_MATLABscripts\PUPILS-preprocessing-pipeline-master-Helia\data'\
            #           '_baselineQ.csv', index=None, header=None)
            #saccadeAvgVelocity, saccadePeakVelocity = saccadeDetection_mDBSCAN(gazePoint_cm, gazePoint_degrees)
            
            """
            
            if '1stPart' in root:
                print('1stPart')
                saccadeAmplitude1 = saccadeAmplitude
                saccadeDuration1 = saccadeDuration
                saccadeVelocity1 = saccadeVelocity
                saccadeTimeStamp1 = saccadeTimeStamp
                
                saccadeAvgAmplitude1 = saccadeAvgAmplitude
                saccadeAvgDuration1 = saccadeAvgDuration
                saccadeAvgVelocity1 = saccadeAvgVelocity
                saccadePeakVelocity1 = saccadePeakVelocity
                continue
                
            if '2ndPart' in root:
                print('2ndPart')
                saccadeAmplitude2 = saccadeAmplitude
                saccadeDuration2 = saccadeDuration
                saccadeVelocity2 = saccadeVelocity
                saccadeTimeStamp2 = saccadeTimeStamp
            
                saccadeAvgAmplitude2 = saccadeAvgAmplitude1
                saccadeAvgDuration2 = saccadeAvgDuration
                saccadeAvgVelocity2 = saccadeAvgVelocity
                saccadePeakVelocity2 = saccadePeakVelocity
                
                saccadeAvgAmplitude = saccadeAvgAmplitude1 + saccadeAvgAmplitude2
                saccadeAvgDuration = saccadeAvgDuration1 + saccadeAvgDuration2
                saccadeAvgVelocity = saccadeAvgVelocity1 + saccadeAvgVelocity2
                saccadePeakVelocity = saccadePeakVelocity1 + saccadePeakVelocity2
                
                
                saccadeAmplitude = saccadeAmplitude1 + saccadeAmplitude2
                saccadeDuration = saccadeDuration1 + saccadeDuration2
                saccadeVelocity = saccadeVelocity1 + saccadeVelocity2
                saccadeTimeStamp = saccadeTimeStamp1 + saccadeTimeStamp2
                
                saccadeAmplitude1, saccadeDuration1, saccadeVelocity1, saccadeTimeStamp1 = list(), list(), list(), list()
                saccadeAvgAmplitude1, saccadeAvgDuration1, saccadeAvgVelocity1, saccadePeakVelocity1 = list(), list(), \
                list(), list()
                
                
            dataToSave = DataForEverySubject()
            dataToSave.timeStamp = saccadeTimeStamp
            dataToSave.saccadeAmplitude = saccadeAmplitude
            dataToSave.saccadeDuration = saccadeDuration
            dataToSave.saccadeVelocity = saccadeVelocity
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            """
            """
            dataToSave_average = DataForEverySubject_Average()
            dataToSave_average.saccadeAmplitude = saccadeAvgAmplitude
            dataToSave_average.saccadeDuration = saccadeAvgDuration
            dataToSave_average.saccadeVelocity = saccadeAvgVelocity
            dataToSave_average.saccadePeakVelocity = saccadePeakVelocity
            dataToSave_average.resultPathName = resultFileName_average
            
            #print(dataToSave_average.printInfo())
            """
            
            #dataToSave.AddToFile()
            #dataToSave_average.AddToFile()
            

subject path E:\Data\Data\ac\1\2019-02-11-11-18-30_1
subject and session name:  ac__1__2019-02-11-11-18-30_1
2.280751537025477
2.28165295257378
2.2795390526236177
2.2757073853412355
2.2774883660041803
2.2783357441363643
2.280172728841242
2.2792819535413704
2.280199767496367
2.2787083692618633
2.2791940214962914
2.2771446785986043
2.2821523913871116
2.282718556476336
2.2830727660874643
2.279790058116092
2.2798404128510095
2.2755076506091414
2.2334997052869205
2.2162773786799215
2.1840659064838626
2.1542362891263283
2.1073412281044925
1.97866233166193
1.9526934120897141
1.9399374186749665
1.940916101272925
1.9458312190604248
1.9467824057098895
1.9464035041134633
1.9423572374104754
1.9402060586371566
1.9470420025872666
1.9465603058385097
1.9511168977171303
1.9466721927511912
1.9455090356616391
1.9334989669101312
1.9347451837821037
1.9397218778960619
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
n

2.001882525146902
1.9993742294431056
1.9985411847170387
1.9997915923245722
1.9991622378630731
1.9973843147680013
1.9979985015150123
2.0001979686586284
1.9987416418581485
1.9979530599325674
1.9964124281390958
1.9974193274229615
1.9982981189520725
1.9953571431850354
1.9944906118240582
1.9942384213979725
1.9970242177968707
1.9982329916305845
1.9955765106629717
1.9953390538700595
1.9939535279501075
1.9991345483439402
2.000205102089145
2.000368790096301
1.9990070930567474
1.999163397591362
1.999302662337735
1.99723096003907
1.9965230770796556
1.9978370550957372
2.000999096106834
2.0022600878769055
2.000304845845957
1.9976347542387571
1.995613808192851
1.9983675039115762
2.0002278186470024
2.0048461695233186
2.001021129073377
2.0011955791494005
2.004472436587765
2.0071382571449403
2.0088575349372055
2.0065016769401587
2.006409354729988
2.0053461564414694
2.0068459613875254
2.008554594767421
2.011168757795489
2.0090211899869987
2.015109700020754
2.0341229222218313
2.036257033615863
2.03086917

1.7958874053836897
1.7962225892688284
1.7959636126673775
1.7888526349440472
1.7860896263941184
1.786112353935755
1.7885927810898639
1.7904608597694263
1.7822757315449989
1.782267679169375
1.779614034490951
1.781354356809401
1.7825677874703312
1.7816127943695361
1.783176070405979
1.7789150761341703
1.7746148893282776
1.7747906227303414
1.7768686949431893
1.7751935640651773
1.7713370491750318
1.7730026613136474
1.7759572493316644
1.7863071558111037
1.804474507628842
1.8113028198155312
1.808115348972786
nan
1.790838479817631
1.7772436321582135
1.7635718321745357
1.7562775367084063
nan
1.7316123694810104
1.7239307256520782
1.7222215265584515
1.7186187807402225
1.7178073811448247
1.7204180516712848
1.7181919278814575
1.717972110538514
1.7162011146711749
1.7163841505560473
1.7158212589572057
1.7156190107904332
1.716181369328788
1.7136638655510097
1.7125790739950084
1.7093880166125552
1.710800559704337
1.7111435191880118
1.7115013924116045
1.712757486502476
1.7127475066642686
1.71415324433058

2.0275603710990393
2.026159909498157
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.9430558145532775
2.913583286355551
2.956080705650294
2.900655765574346
2.9029577318519224
2.863381155710475
2.882024256387408
2.864865289471697
nan
2.9966315197260056
3.069450634726767
3.0675403383969937
3.070896841582112
nan
2.931940361099679
2.8306250685794256
2.702677495583326
2.6806421852593636
2.658414628551435
2.672527006524533
2.7265901920588926
2.8634357662973087
2.8536835035788273
2.8819492879

2.5215038464566266
2.522996092688976
2.521834394178762
2.5226662927991885
2.5197773807386565
2.520586149118775
2.5164798984723893
2.5194754415374327
2.5232383517364037
2.5470503435848917
2.5843457665777674
2.6210712000522287
2.63536057776617
2.642835128090975
2.607718922502793
2.535773646225273
2.4456536858086486
2.41838416161702
2.4069772999030277
2.395461105226524
2.3905572552640915
2.3884676245551324
2.3920224511834443
2.3933438491848027
2.3966344511696924
2.395559642995706
2.395227905056735
2.3909154415034166
2.3881319856967376
2.3900763880713036
2.3888204658933376
2.3901686053331996
2.385059682838054
2.382963244241339
2.381626290792379
2.3937772349838413
2.3994731632770585
2.403804521211805
2.3993012211426255
2.4003790909932574
2.3978297711722747
2.394751829447897
2.394946415980934
2.3956072359530878
2.396235465282599
2.3987998488615974
2.424421029299256
2.5261708755264705
2.5471664312476205
2.5692910827751883
2.5877098698623895
2.5349990830194953
2.3809667294704755
2.340986198935

...

1.799384480867756
1.7999127004789452
1.7988595037083457
1.799863383882055
1.7994645786965349
1.7997371961473942
1.8007309889033327
1.8016850643822997
1.8014786473387405
1.8022229562086656
1.8025499327029852
1.803631023414582
1.808261242722096
1.8074180887835638
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1.8560631602018398
1.8312569180976945
1.8138160583993552
1.7851317006737775
1.7876755036070622
1.797090464081593
1.798014574860738
1.7957643975495325
1.7930907249271932
1.7928264929318387
1.7916785714212173
1.793651561671044
1.7921802831787894
1.7934139067710158
1.7921057261004336
1.7922630511830733
1.79301931325441
1.7935156091484392
1.7958813348633893
1.7956087308842812
1.7980018902028938
1.7965901312291548
1.796920070344569
1.796393726202261
1.7962390235661985
1.7987670298153953
1.798845566897419
1.7984644804798522
1.7978635453680907
1.7966055370963732
1.7962426249722305
1.8024456033391107
1.80570465434095
1.8181896150585113
1.8226184244361945
1.8232564214793

2.230671919463093
2.2283793379626555
2.2264351537678233
2.2250578262468914
2.2270285258826945
2.231432349200779
2.235294204692931
2.2340803154476667
2.2322444371471595
2.2312887481940327
2.2286293593356916
2.2311116470067884
2.229800769171802
2.2330035086511315
2.2311968426500557
2.2341238083818524
2.232390821147413
2.232246003882969
2.2293765657137667
2.2291079517465135
2.228773104261886
2.227509643713867
2.2290728483295728
2.227172855980239
2.225104727638715
2.2244418153571797
2.2275527665659576
2.2276927170072547
2.2281783375458786
2.227384185915589
2.228110341526716
2.22751645505488
2.224939653778362
2.2238987941007085
2.223591031253952
2.221110865207629
2.22300152914853
2.2228021888868432
2.221639998194242
2.2225068911998997
2.224266991499366
2.2262417799053016
2.2269905090376505
2.2252536793095885
2.2236290831371823
2.220572854253602
2.2215323016030606
2.2226675383992847
2.2230777563217634
2.221849501835186
2.222994687650024
2.2259880756632033
2.227058368896002
2.231603307055428


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
1.6058832156399194
1.6038422173363378
1.599079958412976
1.6072534296609573
1.6104027014922693
1.6075789818404453
1.6003259474468807
1.59103847223815
1.599482272147508
1.603723863283321
1.6055225291598525
1.6021979943108626
1.6083756089731702
nan
1.6104426454472693
1.6131366518038428
1.6402411060862288
1.731609655431315
nan
1.7450192763496128
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.16198528431394
nan
nan
nan
nan
2.2737994215810753
2.276896413577326
2.2757634255429653
2.2615681770358464
2.2522831927970004
2.2406589783035034
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.356391070999061
2.2838327089826915
2.2113966412937924
1.9391323463042516
1.8791105437765756
1.7685352887061079
1.7279348

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
2.823265519313688
2.8017923902118125
2.7703007620285893
2.729251648155315
2.6745215176232526
2.661268866846484
2.6631268061022118
2.677016239395986
2.6788192729811713
2.676619768240486
2.6752782169673743
2.671819046008145
2.6692237969392556
2.662755490337387
2.6655486394150216
2.677167911696327
2.6881745192508757
2.6958264417232445
2.69629958486394
2.68996422391551
2.6854629236894523
2.684925688682746
2.685185903212585
2.6797499074495703
2.6775531676366024
2.6782731168422065
2.680788474133245
2.677727011406322
2.6789780825782907
2.682015622704887
2.685683291936909
2.6838885660399625
2.6851058987864738
2.68822389869357
2.6912328787747795
2.69127248627312
2.690814606945968
2.6898861325322754
2.6901701981618475
2.6862607165501484
2.6859384320252113
2.6851105428764006
2.6826434159401478
2.6799650237607127
2.6797344176080236
2.683035711074167
2.6820230529931783
2.6820529275906777
2.682772111033586
2.68579486008658
2.68576988983

NameError: name 'fs' is not defined

In [154]:
math.acos(0.28)

1.2870022175865687

# open interactive plot of saccade velocity

In [ ]:
import numpy as np
b = list()

b.append([(1, 2, 3), 'b'])
b.append([(1, 2, 3), 'c'])

In [ ]:
np.array(b)

In [ ]:
b[0]

In [ ]:
c = np.array([[b[i][0], b[i][1], ''] for i in range(len(b))])

In [ ]:
c[0][2] = 'h'

In [ ]:
c[0,1]